In [65]:
import pandas as pd
import csv
from google.colab import files
import numpy as np

In [66]:
files.upload()
archivo_csv = 'ventas.csv'

#leer archivo .csv
with open(archivo_csv, 'r', encoding='utf-8') as f:
    sample = f.read(2048)
    dialect = csv.Sniffer().sniff(sample)

df_ventas = pd.read_csv(archivo_csv, sep=dialect.delimiter)
print(df_ventas)


Saving ventas.csv to ventas.csv
    Cliente_ID      Nombre    Categoría  Cantidad Precio      Método de Pago  \
0         1001   Cliente_1  Electrónica         2   45,5              PayPal   
1         1002   Cliente_2         Moda         1     30  Tarjeta de Crédito   
2         1003   Cliente_3        Hogar         3  25,75            Efectivo   
3         1004   Cliente_4     Deportes         4     50  Tarjeta de Crédito   
4         1005   Cliente_5          NaN         2     40              PayPal   
5         1006   Cliente_6  Electrónica         6  75,25            Efectivo   
6         1007   Cliente_7  Electrónica        50     90  Tarjeta de Crédito   
7         1008   Cliente_8     Deportes        60     85            Efectivo   
8         1009   Cliente_9         Moda         5   32,5              PayPal   
9         1010  Cliente_10        Hogar         7     95            Efectivo   
10        1011  Cliente_11         Moda         3    NaN            Efectivo   
11      

In [67]:
# Identificar y manejar valores perdidos
print("\nNULOS\n")
print(df_ventas.isnull())
print("\nSUMA DE NULOS\n")
print(df_ventas.isnull().sum())
datos_rellenos = df_ventas.copy()

#reemplaza datos
datos_rellenos["Categoría"] = datos_rellenos["Categoría"].fillna(datos_rellenos["Categoría"].mode()[0])
datos_rellenos["Precio"] = datos_rellenos["Precio"].str.replace(",", ".")
datos_rellenos["Precio"] = pd.to_numeric(datos_rellenos["Precio"], errors='coerce')
datos_rellenos["Cantidad"] = datos_rellenos["Cantidad"].fillna(0)

#calcular el promedio de la columna Precio por Categoria
datos_rellenos["Promedio_Precio_Categoría"] = datos_rellenos.groupby("Categoría")["Precio"].transform("mean")
datos_rellenos["Precio"] = datos_rellenos["Precio"].fillna(datos_rellenos["Promedio_Precio_Categoría"])
datos_rellenos.drop("Promedio_Precio_Categoría", axis=1, inplace=True)


print("\Tabla final\n")
print(datos_rellenos)





NULOS

    Cliente_ID  Nombre  Categoría  Cantidad  Precio  Método de Pago  Fecha
0        False   False      False     False   False           False  False
1        False   False      False     False   False           False  False
2        False   False      False     False   False           False  False
3        False   False      False     False   False           False  False
4        False   False       True     False   False           False  False
5        False   False      False     False   False           False  False
6        False   False      False     False   False           False  False
7        False   False      False     False   False           False  False
8        False   False      False     False   False           False  False
9        False   False      False     False   False           False  False
10       False   False      False     False    True           False  False
11       False   False       True     False   False           False  False
12       False   

In [68]:
#Detectar y eliminar registros duplicados (2 puntos)
datos_rellenos.drop_duplicates(inplace=True)
print("\nTabla final\n")
print(datos_rellenos)


Tabla final

    Cliente_ID      Nombre    Categoría  Cantidad  Precio      Método de Pago  \
0         1001   Cliente_1  Electrónica         2   45.50              PayPal   
1         1002   Cliente_2         Moda         1   30.00  Tarjeta de Crédito   
2         1003   Cliente_3        Hogar         3   25.75            Efectivo   
3         1004   Cliente_4     Deportes         4   50.00  Tarjeta de Crédito   
4         1005   Cliente_5  Electrónica         2   40.00              PayPal   
5         1006   Cliente_6  Electrónica         6   75.25            Efectivo   
6         1007   Cliente_7  Electrónica        50   90.00  Tarjeta de Crédito   
7         1008   Cliente_8     Deportes        60   85.00            Efectivo   
8         1009   Cliente_9         Moda         5   32.50              PayPal   
9         1010  Cliente_10        Hogar         7   95.00            Efectivo   
10        1011  Cliente_11         Moda         3   31.25            Efectivo   

       Fecha 

In [70]:
print("\n\033[4mDatos Sin Filtrar:\033[0m")
print(datos_rellenos[["Categoría", "Cantidad"]])


# Calcular los cuartiles
primer_cuartil =np.percentile(datos_rellenos["Cantidad"],25)
mediana =np.percentile(datos_rellenos["Cantidad"],50)
tercer_cuartil =np.percentile(datos_rellenos["Cantidad"],75)

# Calcular el rango intercuartílico (IQR)
rango_intercuartil = tercer_cuartil - primer_cuartil
limite_inferior = primer_cuartil - 1.5 * rango_intercuartil
limite_superior = tercer_cuartil + 1.5 * rango_intercuartil

#Filtrar los datos dentro del rango permitido
datos_rellenos = datos_rellenos[(datos_rellenos["Cantidad"] >= limite_inferior) & (datos_rellenos["Cantidad"] <= limite_superior)]


#Mostrar resultados
print("\n\033[4mCuartiles:\033[0m")
print(f"Primer cuartil (Q1, 25%): $ {primer_cuartil:.2f}")
print(f"Mediana  (Q2, 50%): $ {mediana:.2f}")
print(f"Tercer cuartil  (Q3, 75%): $ {tercer_cuartil:.2f}")
print(f"\n\033[4mRango Permitido: $\033[0m {limite_inferior:.2f} - {limite_superior:.2f}")
print(f"\n\033[4mTransacciones dentro del rango permitido:\033[0m")
print(datos_rellenos)



Datos Sin Filtrar:
      Categoría  Cantidad
0   Electrónica         2
1          Moda         1
2         Hogar         3
3      Deportes         4
4   Electrónica         2
5   Electrónica         6
8          Moda         5
9         Hogar         7
10         Moda         3

Cuartiles:
Primer cuartil (Q1, 25%): $ 2.00
Mediana  (Q2, 50%): $ 3.00
Tercer cuartil  (Q3, 75%): $ 5.00

Rango Permitido: $ -2.50 - 9.50

Transacciones dentro del rango permitido:
    Cliente_ID      Nombre    Categoría  Cantidad  Precio      Método de Pago  \
0         1001   Cliente_1  Electrónica         2   45.50              PayPal   
1         1002   Cliente_2         Moda         1   30.00  Tarjeta de Crédito   
2         1003   Cliente_3        Hogar         3   25.75            Efectivo   
3         1004   Cliente_4     Deportes         4   50.00  Tarjeta de Crédito   
4         1005   Cliente_5  Electrónica         2   40.00              PayPal   
5         1006   Cliente_6  Electrónica         6   